In [ ]:
%pip install optuna

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
from sklearn.decomposition import PCA
from sklearn.feature_selection import mutual_info_regression
from scipy.stats import entropy

import tensorflow as tf
from tensorflow.keras import layers, models

import librosa

import pickle
import optuna

# read data from google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def load_audio_files(base_directory, dataset='train'):
    categories = ['fan', 'gearbox', 'pump', 'slider', 'ToyCar', 'ToyTrain', 'valve']
    base_directories = [base_directory + '/' + category + '/' for category in categories]
    audio_files = []

    for i, base_directory in enumerate(base_directories):
        category = categories[i]
        for root, _, files in os.walk(base_directory):
            for filename in files:
                if filename.endswith('.wav'):
                    filepath = os.path.join(root, filename)
                    audio_files.append(filepath)
                    audio, sr = librosa.load(filepath, sr=None)
                    section = filename[8:10]
                    if 'normal' in filename:
                        audio_files.append((audio, sr, category, dataset, 'normal', section))
                    elif 'anomaly' in filename:
                        audio_files.append((audio, sr, category, dataset, 'anomaly', section))
    return audio_files

base_directory = '/content'
train_audio_files = load_audio_files(base_directory, dataset='train')

In [ ]:
machine = ['fan', 'gearbox', 'pump', 'slider', 'ToyCar', 'ToyTrain']
# Specify the filename of the pickle file
# for mel-spectrogram
f1 = '/content/drive/MyDrive/path_to_file.pkl'
with open(f1, 'rb') as file:
    train_data = pickle.load(file)

f2 = '/content/drive/MyDrive/path_to_file.pkl'
with open(f2, 'rb') as file:
    train_label = pickle.load(file)

f3 = '/content/drive/MyDrive/path_to_file.pkl'
with open(f3, 'rb') as file:
    test_data = pickle.load(file)

f4 = '/content/drive/MyDrive/path_to_file.pkl'
with open(f4, 'rb') as file:
    test_label = pickle.load(file)

# for idnn
f5 = '/content/drive/MyDrive/path_to_file.pkl'
with open(f5, 'rb') as file:
    y_train = pickle.load(file)

f6 = '/content/drive/MyDrive/path_to_file.pkl'
with open(f6, 'rb') as file:
    y_test = pickle.load(file)

f7 = '/content/drive/MyDrive/path_to_file.pkl'
with open(f7, 'rb') as file:
    X_train_indices = pickle.load(file)

f8 = '/content/drive/MyDrive/path_to_file.pkl'
with open(f8, 'rb') as file:
    X_test_indices = pickle.load(file)

In [ ]:

# Function to compute mutual information between original waveforms and log mel spectrograms
def compute_mutual_information(waveforms, log_mel_spectrograms):
    mi_scores = []
    for i in range(waveforms.shape[0]):
    # Average pooling of the spectrogram to match the waveform length
        pooled_spectrogram = log_mel_spectrograms[i].mean(axis=0)
        pooled_spectrogram = pooled_spectrogram.flatten()
        if len(pooled_spectrogram) != len(waveforms[i]):
    # Downsample waveform to match pooled spectrogram length
            downsampled_waveform = librosa.resample(waveforms[i], orig_sr=len(waveforms[i]), target_sr=len(pooled_spectrogram))
        else:
            downsampled_waveform = waveforms[i]

        mi = mutual_info_regression(pooled_spectrogram.reshape(-1, 1), downsampled_waveform)
        mi_scores.append(mi.mean())
    return np.mean(mi_scores)


# Function to compute Shannon entropy of the original waveforms and log mel spectrograms
def compute_entropy(data):
    entropies = []
    for sample in data:
        # Normalize the sample to avoid log(0)
        sample = sample - np.min(sample)
        sample = sample / np.max(sample)
        ent = entropy(sample)
        entropies.append(ent)
    return np.mean(entropies)

log_mel_spectrograms = np.array(features_list)
# Compute mutual information
mi_waveform_vs_spectrogram = compute_mutual_information(waveforms, log_mel_spectrograms)
print(f"Average Mutual Information: {mi_waveform_vs_spectrogram}")

# Compute entropy for original waveforms and spectrograms
entropy_waveforms = compute_entropy(waveforms)
entropy_spectrograms = compute_entropy(log_mel_spectrograms.reshape(B, -1))

print(f"Average Entropy of Waveforms: {entropy_waveforms}")
print(f"Average Entropy of Spectrograms: {entropy_spectrograms}")


numpy.ndarray

In [ ]:
# read the labels
# train_labels_df = pd.DataFrame(train_label, columns=['machine_type', 'source', 'normal_anomaly', 'section'])
test_labels_df = pd.DataFrame(test_label, columns=['machine_type', 'source', 'normal_anomaly', 'section'])

section = test_labels_df['section'].unique()
source = test_labels_df['source'].unique()
machine_type = test_labels_df['machine_type'].unique()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# train_labels_df[train_labels_df['machine_type'] == 'fan']
# test_labels_df
machine_type

array(['valve'], dtype=object)

In [ ]:
def get_train_data_machine(machine_type):

    train_indices = train_labels_df[train_labels_df['machine_type'] == machine_type].index
    test_indices = test_labels_df[test_labels_df['machine_type'] == machine_type].index

    train_data_filtered = train_data[train_indices]
    test_data_filtered = test_data[test_indices]
    test_labels_filtered = test_labels_df.loc[test_indices, 'normal_anomaly'].values
    test_labels_filtered = np.array([1 if x == 'anomaly' else 0 for x in test_labels_filtered])
    test_category_filtered = test_labels_df.loc[test_indices, ['source', 'section']].reset_index().drop(columns='index')

    return train_data_filtered, test_data_filtered, test_labels_filtered, test_category_filtered

def get_labels_machine_idnn(machine_type):

    train_indices = train_labels_df[train_labels_df['machine_type'] == machine_type].index
    test_indices = test_labels_df[test_labels_df['machine_type'] == machine_type].index

    test_labels_filtered = test_labels_df.loc[test_indices, 'normal_anomaly'].values
    test_labels_filtered = np.array([1 if x == 'anomaly' else 0 for x in test_labels_filtered])
    test_category_filtered = test_labels_df.loc[test_indices, ['source', 'section']].reset_index().drop(columns='index')

    return test_labels_filtered, test_category_filtered

def get_label(train_data, test_data, test_labels_df):

    train_data_filtered = np.array(train_data)
    test_data_filtered = np.array(test_data)
    test_labels_filtered = test_labels_df['normal_anomaly'].values
    test_labels_filtered = np.array([1 if x == 'anomaly' else 0 for x in test_labels_filtered])
    test_category_filtered = test_labels_df[['source', 'section']]

    return train_data_filtered, test_data_filtered, test_labels_filtered, test_category_filtered




In [ ]:
# for idnn only
def extract_patches_with_tracking(data, patch_size=3):
    patches = []
    labels = []
    indices = []
    for b in range(data.shape[0]):
        sample = data[b]
        for i in range(1, sample.shape[0]-1):
            for j in range(1, sample.shape[1]-1):
                patch = sample[i-1:i+2, j-1:j+2]
                patches.append(np.delete(patch.flatten(), 4))  # Remove the center element
                labels.append(patch[1, 1])  # Center element as the label
                indices.append((b, i, j))  # Track (spectrogram index, row, col)
    return np.array(patches), np.array(labels), indices

X_train, Y_train, indices_train = extract_patches_with_tracking(train_data)
X_train = X_train.reshape(-1, 8)
X_test, Y_test, indices_test = extract_patches_with_tracking(test_data)
X_test = X_test.reshape(-1, 8)

def get_label(test_labels_df):

    test_labels_filtered = test_labels_df['normal_anomaly'].values
    test_labels_filtered = np.array([1 if x == 'anomaly' else 0 for x in test_labels_filtered])
    test_category_filtered = test_labels_df[['source', 'section']]

    return test_labels_filtered, test_category_filtered

In [ ]:
# define auto encoder model
def create_autoencoder(data, units_layer_1, units_layer_2, units_layer_3, units_layer_4, latent_dim):

    input_shape = (data.shape[1], data.shape[2])
    # Create the input layer
    input_layer = layers.Input(shape=input_shape)

    # Flatten the input if you are using dense layers for encoding
    flattened_input = layers.Flatten()(input_layer)

    #Encoder block
    x = layers.Dense(units_layer_1, activation="relu")(flattened_input)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(units_layer_2, activation="relu")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(units_layer_3, activation="relu")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(units_layer_4, activation="relu")(x)
    x = layers.BatchNormalization()(x)
    #Latent space
    x = layers.Dense(latent_dim, activation="relu")(x)
    x = layers.BatchNormalization()(x)

    #Decoder block
    x = layers.Dense(units_layer_4, activation="relu")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(units_layer_3, activation="relu")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(units_layer_2, activation="relu")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(units_layer_1, activation="relu")(x)
    x = layers.BatchNormalization()(x)
    #Output layer
    x = layers.Dense(input_shape[0]*input_shape[1], activation=None)(x)
    output_layer = layers.Reshape(input_shape)(x)

    # Create and return the model
    autoencoder = models.Model(inputs=input_layer, outputs=output_layer)

    return autoencoder

# Data Preparation
def prepare_dataloader(data, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices(data)
    dataset = dataset.batch(batch_size).shuffle(buffer_size=len(data))
    return dataset

# Training Function
def train_model(model, dataloader, epochs, optimizer, loss_fn):
    for epoch in range(epochs):
        print(f'Epoch {epoch+1}/{epochs}')
        for batch in dataloader:
            with tf.GradientTape() as tape:
                reconstruction = model(batch, training=True)
                loss = loss_fn(batch, reconstruction)
            gradients = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        print(f'Loss: {loss.numpy()}')


def objective(trial, data_train, data_test):
    # Suggest the number of units for each hidden layer
    units_layer_1 = trial.suggest_int('units_layer_1', 320, 512)
    units_layer_2 = trial.suggest_int('units_layer_2', 256, 320)
    units_layer_3 = trial.suggest_int('units_layer_3', 128, 256)
    units_layer_4 = trial.suggest_int('units_layer_4', 64, 128)
    latent_dim = trial.suggest_int('latent_dim', 8, 32)

    # Suggest the learning rate and batch size
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-2)
    batch_size = trial.suggest_categorical('batch_size', [128, 256])

    # Suggest the number of epochs
    epochs = trial.suggest_int('epochs', 100, 300)

    # Create the autoencoder model with the suggested parameters
    autoencoder = create_autoencoder(data_train, units_layer_1, units_layer_2, units_layer_3, units_layer_4, latent_dim)

    # Define the optimizer and loss function
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    loss_fn = tf.keras.losses.MeanSquaredError()

    # Prepare the data loader
    train_loader = prepare_dataloader(data_train, batch_size)

    # Train the model
    train_model(autoencoder, train_loader, epochs, optimizer, loss_fn)

    # Evaluate on validation set
    reconstruction = autoencoder.predict(data_test)
    loss = loss_fn(data_test, reconstruction).numpy()

    return loss

def calculate_ae_metrics(mse, test_labels_filtered, test_category_filtered):
    results = []

    for s in section:
        for so in source:
            idx = test_category_filtered[ (test_category_filtered['source'] == so ) &
                                        (test_category_filtered['section'] == s)
                                        ].index

            mse_subset = mse[idx]
            test_labels_subset = test_labels_filtered[idx]

            anomaly_percent_subset = np.sum(test_labels_subset) / test_labels_subset.shape[0] * 100
            print(f'the percentage of anomaly from section {s} of {so} is {anomaly_percent_subset:.2f} %')

            threshold_subset = np.percentile(mse_subset, 100-anomaly_percent_subset)
            # threshold_subset = np.percentile(mse_subset, 55)
            anomalies_subset = np.array([1 if x > threshold_subset else 0 for x in mse_subset])

            auc = roc_auc_score(test_labels_subset, mse_subset)
            p_auc = roc_auc_score(test_labels_subset, mse_subset, max_fpr=0.1)
            f1 = f1_score(test_labels_subset, anomalies_subset)

            results.append([s, so, anomaly_percent_subset, threshold_subset, auc, p_auc, f1])

    return results



In [ ]:
# define the objective functions for each model for hyper parameter tuning

def create_cnn_autoencoder(data, filter1, filter2, filter3, kernel_size1, kernel_size2, kernel_size3):
    input_shape = (data.shape[1], data.shape[2], 1)
    input_layer = layers.Input(shape=input_shape)  # (B, 28, 640, 1)

    # Encoder
    x = layers.Conv2D(filter1, kernel_size1, activation='relu', padding='same')(input_layer)  # (B, 28, 640, 1) -> (B, 28, 640, 64)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)  # (B, 14, 320, 64)
    x = layers.Conv2D(filter2, kernel_size2, activation='relu', padding='same')(x)  # (B, 14, 320, 64) -> (B, 14, 320, 32)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)  # (B, 7, 160, 32)
    x = layers.Conv2D(filter3, kernel_size3, activation='relu', padding='same')(x)  # (B,  7, 160, 32) -> (B,  7, 160, 16)
    encoded = layers.MaxPooling2D((1, 2), padding='same')(x)  # (B,  4, 80, 16)

    # Decoder
    x = layers.Conv2DTranspose(filter3, kernel_size3, activation='relu', padding='same')(encoded)
    x = layers.UpSampling2D((1, 2))(x)
    x = layers.Conv2DTranspose(filter2, kernel_size2, activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2, 2))(x)
    x = layers.Conv2DTranspose(filter1, kernel_size1, activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2, 2))(x)
    decoded = layers.Conv2D(1, kernel_size1, activation='sigmoid', padding='same')(x)

    cnn = models.Model(inputs=input_layer, outputs=decoded)

    return cnn


def prepare_dataloader(data, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices(data)
    dataset = dataset.batch(batch_size).shuffle(buffer_size=len(data))
    return dataset

# Training Function
def train_model(model, dataloader, epochs, optimizer, loss_fn):
    for epoch in range(epochs):
        print(f'Epoch {epoch+1}/{epochs}')
        for batch in dataloader:
            with tf.GradientTape() as tape:
                reconstruction = model(batch, training=True)
                loss = loss_fn(batch, reconstruction)
            gradients = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        print(f'Loss: {loss.numpy()}')

# def objective_cnn(trial, data_train, data_test, data_train_expanded, data_test_expanded):
def objective_cnn(trial, data_train, data_test):

    filter1 = trial.suggest_int('filter1', 64, 128)
    filter2 = trial.suggest_int('filter2', 16, 64)
    filter3 = trial.suggest_int('filter3', 4, 16)
    kernel_size1 = trial.suggest_int('kernel_size1', 3, 5)
    kernel_size2 = trial.suggest_int('kernel_size2', 3, 5)
    kernel_size3 = trial.suggest_int('kernel_size3', 3, 5)

    # Suggest the learning rate and batch size
    lr = trial.suggest_loguniform('lr', 1e-4, 1e-2)
    batch_size = trial.suggest_categorical('batch_size', [64, 96])

    # Suggest the number of epochs
    epochs = trial.suggest_int('epochs', 100, 300)

    # Create the autoencoder model with the suggested parameters
    cnn = create_cnn_autoencoder(data_train, filter1, filter2, filter3, kernel_size1, kernel_size2, kernel_size3)

    # Define the optimizer and loss function
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    loss_fn = tf.keras.losses.MeanSquaredError()

    # Prepare the data loader
    train_loader = prepare_dataloader(data_train, batch_size)


    # Train the model
    train_model(cnn, train_loader, epochs, optimizer, loss_fn)


    reconstruction = cnn.predict(data_test)
    loss = np.mean(np.mean(np.square(reconstruction - data_test), axis=(1, 2, 3)))

    print('val_loss=', loss)
    return loss

def calculate_cnn_metrics(mse, test_labels_filtered, test_category_filtered):
    results = []
    for s in section:
        for so in source:
            idx = test_category_filtered[ (test_category_filtered['source'] == so ) &
                                        (test_category_filtered['section'] == s)
                                        ].index

            mse_subset = mse[idx]
            test_labels_subset = test_labels_filtered[idx]

            anomaly_percent_subset = np.sum(test_labels_subset) / test_labels_subset.shape[0] * 100
            print(f'the percentage of anomaly from section {s} of {so} is {anomaly_percent_subset:.2f} %')

            threshold_subset = np.percentile(mse_subset, 100-anomaly_percent_subset)
            anomalies_subset = np.array([1 if x > threshold_subset else 0 for x in mse_subset])

            auc = roc_auc_score(test_labels_subset, mse_subset)
            p_auc = roc_auc_score(test_labels_subset, mse_subset, max_fpr=0.1)
            f1 = f1_score(test_labels_subset, anomalies_subset)

            results.append([s, so, anomaly_percent_subset, threshold_subset, auc, p_auc, f1])

    return results


In [ ]:
# # define idnn model

def create_fc_idnn(data, n_layers, units, dropout_rate, learning_rate):
    model = models.Sequential()
    model.add(layers.InputLayer(input_shape=(data.shape[1],)))

    for _ in range(n_layers):
        model.add(layers.Dense(units, activation='relu'))
        model.add(layers.Dropout(dropout_rate))

    model.add(layers.Dense(1))  # Output layer for regression

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mse')

    return model

def create_idnn_model(trial):
    # Suggest hyperparameters
    n_layers = trial.suggest_int("n_layers", 2, 5)
    units = trial.suggest_int("units", 8, 64)
    dropout_rate = trial.suggest_float("dropout_rate", 0.1, 0.5)
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-4, 1e-2)

    model = models.Sequential()
    model.add(layers.InputLayer(input_shape=(8,)))

    for _ in range(n_layers):
        model.add(layers.Dense(units, activation='relu'))
        model.add(layers.Dropout(dropout_rate))

    model.add(layers.Dense(1))  # Output layer for regression

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mse')

    return model


def objective_idnn(trial, x_train, y_train, x_test, y_test, idx_test):
    model = create_idnn_model(trial)

    # Suggest hyperparameters for training
    batch_size = trial.suggest_int("batch_size", 4096, 8192)
    epochs = trial.suggest_int("epochs", 5, 25)

    # Train the model
    model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_test, y_test), verbose=1)

    # Predict and calculate element-wise squared error for each patch
    predictions = model.predict(x_test, batch_size=batch_size)
    element_wise_squared_error = (predictions.reshape(y_test.shape) - y_test) ** 2

    # Aggregate reconstruction error by spectrogram
    error_dict = {}
    for i, (b, row, col) in enumerate(idx_test):
        if b not in error_dict:
            error_dict[b] = []
        error_dict[b].append(element_wise_squared_error[i])

    # Calculate mean squared error for each spectrogram
    mean_errors = {b: np.mean(errors) for b, errors in error_dict.items()}
    mse = np.mean(np.array(list(mean_errors.values())))

    return mse

def calculate_idnn_metrics(mean_errors, test_labels_filtered, test_category_filtered):
    results = []
    for s in section:
        for so in source:
            idx = test_category_filtered[ (test_category_filtered['source'] == so ) &
                                        (test_category_filtered['section'] == s)
                                        ].index


            print(len(idx))
            print(type(mean_errors))
            print(len(mean_errors))
            mse_subset = mean_errors[idx]
            test_labels_subset = test_labels_filtered[idx]


            anomaly_percent_subset = np.sum(test_labels_subset) / test_labels_subset.shape[0] * 100

            print(f'the percentage of anomaly from section {s} of {so} is {anomaly_percent_subset:.2f} %')

            threshold_subset = np.percentile(mse_subset, 100-anomaly_percent_subset)
            anomalies_subset = np.array([1 if x > threshold_subset else 0 for x in mse_subset])

            auc = roc_auc_score(test_labels_subset, mse_subset)
            p_auc = roc_auc_score(test_labels_subset, mse_subset, max_fpr=0.1)
            f1 = f1_score(test_labels_subset, anomalies_subset)

            results.append([s, so, anomaly_percent_subset, threshold_subset, auc, p_auc, f1])

    return results


In [ ]:
def train_and_evaluate(feat, machine_type, model, no_trials):

    from sklearn.metrics import roc_auc_score, f1_score
    if feat == 'spectro':
        # Filter data for the given machine type
        if model == 'idnn':
            test_labels_filtered, test_category_filtered = get_labels_machine_idnn(machine_type)
            # print(len(test_labels_filtered))
            # print(test_category_filtered)
        else:
            train_data_filtered, test_data_filtered, test_labels_filtered, test_category_filtered = get_train_data_machine(machine_type)
    elif feat == 'emb':
        test_labels_filtered, test_category_filtered = get_label(test_labels_df)
        # train_data_filtered, test_data_filtered, test_labels_filtered, test_category_filtered = get_label(train_data, test_data, test_labels_df)

    # set the number of trials
    trials = no_trials

    if model == 'ae':
        study = optuna.create_study(direction='minimize')
        # study.optimize(objective, n_trials=trials)
        study.optimize(lambda trial: objective(trial, train_data_filtered, test_data_filtered), n_trials=trials)
        best_params = study.best_params
        print(best_params)

        # Train Final Model with Best Hyperparameters
        latent_dim = best_params['latent_dim']
        lr = best_params['lr']
        batch_size = best_params['batch_size']
        epochs = best_params['epochs']
        units_layer_1 = best_params['units_layer_1']
        units_layer_2 = best_params['units_layer_2']
        units_layer_3 = best_params['units_layer_3']
        units_layer_4 = best_params['units_layer_4']


        autoencoder = create_autoencoder(train_data_filtered, units_layer_1, units_layer_2, units_layer_3, units_layer_4, latent_dim)
        optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
        loss_fn = tf.keras.losses.MeanSquaredError()

        train_loader = prepare_dataloader(train_data_filtered, batch_size)
        train_model(autoencoder, train_loader, epochs, optimizer, loss_fn)

        autoencoder.save(f'/content/drive/MyDrive/Data Science/Final Project/models/{feat}_ae_{machine_type}.h5')

        # Predict, calculate reconstruction error and calculate metrics
        reconstruction = autoencoder.predict(test_data_filtered)
        mse = np.mean(np.square(reconstruction - test_data_filtered), axis=(1, 2))
        results = calculate_ae_metrics(mse, test_labels_filtered, test_category_filtered)

    elif model == 'cnn':

        train_data_filtered_expanded = np.expand_dims(train_data_filtered, axis=-1)
        test_data_filtered_expanded = np.expand_dims(test_data_filtered, axis=-1)
        print(train_data_filtered_expanded.shape)
        print(test_data_filtered_expanded.shape)

        study = optuna.create_study(direction='minimize')
        study.optimize(lambda trial: objective_cnn(trial, train_data_filtered_expanded, test_data_filtered_expanded), n_trials=trials)

        best_params = study.best_params
        print(best_params)

        # Train Final Model with Best Hyperparameters
        filter1 = best_params['filter1']
        filter2 = best_params['filter2']
        filter3 = best_params['filter3']
        kernel_size1 = best_params['kernel_size1']
        kernel_size2 = best_params['kernel_size2']
        kernel_size3 = best_params['kernel_size3']
        lr = best_params['lr']
        batch_size = best_params['batch_size']
        epochs = best_params['epochs']

        cnn = create_cnn_autoencoder(train_data_filtered_expanded, filter1, filter2, filter3, kernel_size1, kernel_size2, kernel_size3)
        optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
        loss_fn = tf.keras.losses.MeanSquaredError()

        train_loader = prepare_dataloader(train_data_filtered_expanded, batch_size)
        train_model(cnn, train_loader, epochs, optimizer, loss_fn)
        cnn.save(f'/content/drive/MyDrive/Data Science/Final Project/models/{feat}_cnn_{machine_type}.h5')

        # Predict and calculate reconstruction error
        reconstruction = cnn.predict(test_data_filtered_expanded)
        mse = np.mean(np.square(reconstruction - test_data_filtered_expanded), axis=(1, 2, 3))

        results = calculate_cnn_metrics(mse, test_labels_filtered, test_category_filtered)


    elif model == 'idnn':
        study = optuna.create_study(direction="minimize")
        study.optimize(lambda trial: objective_idnn(trial, train_data, y_train, test_data, y_test, X_test_indices), n_trials=trials)

        best_params = study.best_params
        print(best_params)

        # Train Final Model with Best Hyperparameters
        lr = best_params['learning_rate']
        batch_size = best_params['batch_size']
        epochs = best_params['epochs']
        units = best_params['units']
        n_layers = best_params['n_layers']
        dropout_rate = best_params['dropout_rate']

        idnn = create_fc_idnn(train_data, n_layers, units, dropout_rate, lr)
        optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
        loss_fn = tf.keras.losses.MeanSquaredError()

        history = idnn.fit(train_data, y_train, epochs=epochs, batch_size=batch_size, validation_data=(test_data, y_test), verbose=1)

        idnn.save(f'/content/drive/MyDrive/Data Science/Final Project/models/{feat}_idnn_{machine_type}.h5')

        # Predict and calculate element-wise squared error for each patch
        predictions = idnn.predict(test_data, batch_size=batch_size)
        element_wise_squared_error = (predictions.reshape(y_test.shape) - y_test) ** 2

        # Aggregate reconstruction error by spectrogram
        error_dict = {}
        for i, (b, row, col) in enumerate(X_test_indices):
            if b not in error_dict:
                error_dict[b] = []
            error_dict[b].append(element_wise_squared_error[i])

        # Calculate mean squared error for each spectrogram
        mean_errors = {b: np.mean(errors) for b, errors in error_dict.items()}
        mse = np.array(list(mean_errors.values()))

        # calculate the metrics for each section and domain
        results = calculate_idnn_metrics(mse, test_labels_filtered, test_category_filtered)

    else:
        print(f'ValueError: Model {model} is not available!')


    print(f"Machine Type: {machine_type} with {model} + {feat}, metrics calculation completed!" )


    # export the results
    df = pd.DataFrame(results, columns=['section', 'source', 'percentile',
                                        'threshold', 'auc', 'p_auc', 'f1'])
    df.to_csv(f'/content/drive/MyDrive/Data Science/Final Project/data/results_{feat}_{model}_{machine_type}.csv')

In [ ]:
# for machine in machine_type:
for machine in machine_type:
    train_and_evaluate('emb', machine, 'idnn', 50)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Epoch 1/27
16209/16209 ━━━━━━━━━━━━━━━━━━━━ 56s 3ms/step - loss: 0.8324 - val_loss: 0.1064
Epoch 2/27
16209/16209 ━━━━━━━━━━━━━━━━━━━━ 40s 2ms/step - loss: 0.0815 - val_loss: 0.0563
Epoch 3/27
16209/16209 ━━━━━━━━━━━━━━━━━━━━ 40s 2ms/step - loss: 0.0579 - val_loss: 0.0486
Epoch 4/27
16209/16209 ━━━━━━━━━━━━━━━━━━━━ 40s 2ms/step - loss: 0.0547 - val_loss: 0.0524
Epoch 5/27
16209/16209 ━━━━━━━━━━━━━━━━━━━━ 40s 2ms/step - loss: 0.0535 - val_loss: 0.0492
Epoch 6/27
16209/16209 ━━━━━━━━━━━━━━━━━━━━ 40s 2ms/step - loss: 0.0533 - val_loss: 0.0470
Epoch 7/27
16209/16209 ━━━━━━━━━━━━━━━━━━━━ 40s 2ms/step - loss: 0.0532 - val_loss: 0.0488
Epoch 8/27
16209/16209 ━━━━━━━━━━━━━━━━━━━━ 40s 2ms/step - loss: 0.0531 - val_loss: 0.0498
Epoch 9/27
16209/16209 ━━━━━━━━━━━━━━━━━━━━ 40s 2ms/step - loss: 0.0524 - val_loss: 0.0467
Epoch 10/27
16209/16209 ━━━━━━━━━━━━━━━━━━━━ 40s 2ms/step - loss: 0.0523 - val_loss: 0.0484
Epoch 11/27
16209/16209 ━━━━━━━━━━━━━━━━━━━━ 40s 2ms/step - loss: 0.0519 - val_loss: 0.03

6464/6464 ━━━━━━━━━━━━━━━━━━━━ 9s 1ms/step
200
<class 'numpy.ndarray'>
1200
the percentage of anomaly from section 00 of source_test is 50.00 %
200
<class 'numpy.ndarray'>
1200
the percentage of anomaly from section 00 of target_test is 50.00 %
200
<class 'numpy.ndarray'>
1200
the percentage of anomaly from section 01 of source_test is 50.00 %
200
<class 'numpy.ndarray'>
1200
the percentage of anomaly from section 01 of target_test is 50.00 %
200
<class 'numpy.ndarray'>
1200
the percentage of anomaly from section 02 of source_test is 50.00 %
200
<class 'numpy.ndarray'>
1200
the percentage of anomaly from section 02 of target_test is 50.00 %
Machine Type: valve with idnn + emb, metrics calculation completed!
